# Simple Local LLM Application 
LLM Setting:
- LLM 后端服务: LM Studio
- LLM Model: Qwen1.5-7b-chat-Q8_0.gguf

## 1. LLM Interface Setting
LM Studio provide "OpenAI-like API", so I will follow the [quickstart](https://python.langchain.com/docs/get_started/quickstart#building-with-langchain) to set up the llm

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_base="http://localhost:1234/v1",
    openai_api_key="not-needed"
)

Test the llm

In [5]:
resp = llm.invoke("how can langsmith help with testing?")
print(type(resp))
print(resp.content)

<class 'langchain_core.messages.ai.AIMessage'>
Langsmith can be a valuable tool for testing in several ways. Here are some specific ways it can support testing processes:

1. **Code Generation:** Langsmith allows you to generate code snippets or entire libraries in various programming languages. This can save time and effort by automating the process of writing test cases, reducing the risk of human error and ensuring consistency across tests.

2. **Stubbing and Mocking:** By generating mock objects or stubs, Langsmith can help you create realistic testing environments for functions, APIs, or classes. This is particularly useful when testing dependencies that are difficult to set up manually.

3. **Test Case Generation**: With its ability to generate test cases based on input parameters and expected outputs, Langsmith can assist in creating comprehensive test suites. This can be especially helpful for generating negative tests, edge cases, and boundary conditions.

4. **Validation and 

We can see it is working! And langchain return a class named "AIMessage"

## 2. langchain Pipeline

Langchain 还可以使用 “prompt 模板” 对 prompt 进行自定义。prompt template 可以规范 prompt 的格式，使输入更为灵活。

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

我们使用“|”作为“管道”，串联起 prompt 和 llm 这两个组件

In [7]:
chain = prompt | llm

In [8]:
resp = chain.invoke(
    {"input": "请介绍一下langchain"}
)
print(resp.content)

LangChain is a cutting-edge language learning platform that leverages the power of blockchain technology to revolutionize the way people learn and master new languages. At its core, LangChain aims to create a secure, transparent, and decentralized ecosystem for language exchange and education.

1. **Decentralized Learning**: The platform utilizes blockchain's distributed ledger system to store and verify learning records. This ensures that every interaction and accomplishment is permanently recorded and cannot be tampered with, promoting trust among learners and educators.

2. **Digital Certificates**: LangChain provides digital certificates for each language proficiency achieved by its users. These certificates are verifiable and can be easily shared or used for professional purposes, enhancing the value of the learning experience.

3. **Interactive Content**: The platform integrates blockchain-based smart contracts to facilitate interactive content such as immersive language courses,

同样，还可以使用 StrOutputParser 对 LLM 输出进行格式化处理

In [9]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [10]:
chain = prompt | llm | output_parser

In [25]:
chain.invoke(
    {"input": "请介绍一下langchain"}
)

"LangChain is a cutting-edge programming language and its accompanying development framework designed to provide seamless and efficient software development experiences. At the core of LangChain lies a modern, powerful syntax that simplifies code writing and enhances readability, making it ideal for both beginners and advanced developers.\n\nKey Features:\n\n1. **Clean Syntax:** LangChain follows a clean and expressive coding standard, reducing boilerplate and allowing developers to focus on logic rather than syntax.\n\n2. **Type Safety:** The language is type-safe by default, ensuring that errors are caught early and preventing potential bugs during runtime.\n\n3. **Interoperability:** LangChain is designed with extensibility in mind, allowing it to seamlessly integrate with other languages and systems through APIs or libraries.\n\n4. **Object-Oriented**: It supports object-oriented programming principles like inheritance, encapsulation, and polymorphism, making it suitable for buildi

## 3. Retrieval Chain

Langchain 提供 API 方便我们从第三方平台获取（Retrieval）数据，然后交由 LLM 进行分析处理。
接下来将演示通过 Langchain 获取在线网页信息然后交由 LLM 进行处理

In [12]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [22]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://python.langchain.com/docs/get_started/introduction")

docs = loader.load()

下一步，我们将使用本地 Embedding 模型将数据向量化，同时存储在向量数据库中。

第一步：配置本地的 Embeddable 模型。参考 [langchain-how-to-use-a-custom-embedding-model-locally](https://stackoverflow.com/questions/77217193/langchain-how-to-use-a-custom-embedding-model-locally)

In [14]:
# %conda install faiss-gpu

In [23]:
from langchain.embeddings import HuggingFaceEmbeddings

modelPath = "BAAI/bge-base-en-v1.5"

# Create a dictionary with model configuration options, specifying to use the CPU for computations
# model_kwargs = {'device':'cpu'}
model_kwargs = {'device':'cuda'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': True}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

开始向量化数据

In [24]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

现在，我们已经在向量存储中索引了这些数据，我们将创建一个检索链。该链将接受传入的问题，查找相关文档，然后将这些文档与原始问题一起传递到一个LLM中，并要求它回答原始问题。

In [18]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

我们可以直接将文件交由 LLM 处理：

In [19]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'Langsmith can assist in testing by visualizing the results, likely providing a graphical representation or some kind of data visualization that allows users to easily interpret and understand the outcome of the tests. This could be helpful in identifying any issues, trends, or successes in the testing process.'

但是，我们希望文档首先来自我们刚刚设置的检索器。这样，对于给定的问题，我们可以使用检索器动态选择最相关的文档并将其传递。

参考文档：[langchain faiss](https://python.langchain.com/docs/integrations/vectorstores/faiss)

In [20]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [27]:
response = retrieval_chain.invoke({"input": "请介绍一下langchain"})
print(type(response))
print(response)
print(response["answer"])

<class 'dict'>
{'input': '请介绍一下langchain', 'context': [Document(page_content='Introduction | 🦜️🔗 Langchain', metadata={'source': 'https://python.langchain.com/docs/get_started/introduction', 'title': 'Introduction | 🦜️🔗 Langchain', 'description': 'LangChain is a framework for developing applications powered by language models. It enables applications that:', 'language': 'en'}), Document(page_content="and techniques for common end-to-end use cases, like:Document question answeringChatbotsAnalyzing structured dataand much more...Integrations\u200bLangChain is part of a rich ecosystem of tools that integrate with our framework and build on top of it. Check out our growing list of integrations.Guides\u200bBest practices for developing with LangChain.API reference\u200bHead to the reference section for full documentation of all classes and methods in the LangChain and LangChain Experimental Python packages.Developer's guide\u200bCheck out the developer's guide for guidelines on contributing

对比于“直接回答”的结果更为准确、精炼